In [20]:
from cluster_utils import ClustersShots, ClustersTracks

In [17]:
def get_facetracks_and_links(shot_data, th_feats=0.1, th_overlap=0.85):
    fno = 0
    nFaces = len(shot_data)
    ls_inds = [0]

    ls_cannotlink = []
    cl_tracks = ClustersTracks(simThresh=th_overlap, featThresh=th_feats)
    
    for i in range(1, nFaces):
        
        fno = shot_data[i].fno
        if(fno != shot_data[i-1].fno):
            #do something
            ls_data = shot_data[ls_inds]
            ls_cl = cl_tracks.cluster_online(ls_data)
            ls_cannotlink.append(ls_cl)    
            ls_inds = [i]
        else:
            ls_inds.append(i)

    ls_tracks = cl_tracks.clusters
    qMatrix = np.ones((ls_tracks, ls_tracks))

    for l in ls_cannotlink:
        for i in range(len(l)):
            for j in range(i+1, len(l)):
                qMatrix[i,j] = 0
                qMatrix[j,i] = 0
            
    return ls_tracks, qMatrix
    
def process_shot(clusters_shot, ls_frames, detector, aligner, fnet):
    shot_data = extract_bboxes_and_features(ls_frames, detector, aligner, fnet) # list of face_element
    face_tracks, qMatrix = get_facetracks_and_links(shot_data, th)
    clusters_shot.cluster_online(ls_tracks, qMatrix) #the functionwa in paperwa
    

In [3]:
path = "../vids/b_1.mkv"
cap = cv2.VideoCapture(path)
aligner, fnet = initialize_deep_models()
detector = get_dlib_detector()

ret = False

ls_frames = []

for i in range(2):
    ret, frame = cap.read()
    assert(ret==True)
    if(i==0):
        ppframe = frame
    if(i==1):
        pframe = frame
    ls_frames.append(frame)

clusters_shot = ClustersShots(3.0)
    
while(ret == True):
    # Capture frame-by-frame
    ret, frame = cap.read()
    assert(ret==True)
    ls_frames.append(frame)
    # If shot boundary is detected or clip is more than 100 frames (assuming framerate ~20-30fps), process it
    if (shot_boundary(ppframe, pframe, frame) or len(ls_frames) > 100):
        process_shot(clusters_shot, ls_frames, detector, aligner, fnet)
    #update prev-prev frame and prev-frame
    ppframe = pframe
    pframe = frame

# When everything done, release the capture
cap.release()
cv2.destroyAllWindows()

((576, 720, 3), (576, 720, 3))
